In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [4]:
spark = SparkSession.builder \
.appName("datascientistsalaries_dataset") \
.getOrCreate()

In [6]:
df = spark.read \
.option("header",True) \
.option("inferSchema","True") \
.csv("/Users/burak/Desktop/Software Projects/Data Engineering/Data Scientist Salaries/ds_salaries.csv")

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [8]:
#Sort by salary_in_usd in descending order
df.orderBy(F.desc("salary_in_usd")).select("work_year",\
"experience_level","job_title","salary_in_usd",\
"company_location","company_size").show(10)

+---------+----------------+--------------------+-------------+----------------+------------+
|work_year|experience_level|           job_title|salary_in_usd|company_location|company_size|
+---------+----------------+--------------------+-------------+----------------+------------+
|     2021|              EX|Principal Data En...|       600000|              US|           L|
|     2021|              MI|Financial Data An...|       450000|              US|           L|
|     2020|              MI|  Research Scientist|       450000|              US|           M|
|     2021|              MI|Applied Machine L...|       423000|              US|           L|
|     2021|              EX|Principal Data Sc...|       416000|              US|           S|
|     2020|              SE|      Data Scientist|       412000|              US|           L|
|     2022|              SE| Data Analytics Lead|       405000|              US|           L|
|     2022|              SE|Applied Data Scie...|       3800

In [9]:
#count of job_titles
df.select(F.countDistinct("job_title")).alias("Count Of Job Titles").show()

+-------------------------+
|count(DISTINCT job_title)|
+-------------------------+
|                       50|
+-------------------------+



In [10]:
#number of job titles with their types
df.groupBy("job_title").count().orderBy(F.desc("count")).select(col("job_title").alias("Job Title"),col("count").alias("Count")).show(10)


+--------------------+-----+
|           Job Title|Count|
+--------------------+-----+
|      Data Scientist|  143|
|       Data Engineer|  132|
|        Data Analyst|   97|
|Machine Learning ...|   41|
|  Research Scientist|   16|
|Data Science Manager|   12|
|      Data Architect|   11|
|Machine Learning ...|    8|
|   Big Data Engineer|    8|
|Director of Data ...|    7|
+--------------------+-----+
only showing top 10 rows



In [11]:
#longest job titles
df.select(col("job_title").alias("Job Title"),length("job_title").alias("Long Job Title")).orderBy(F.desc("job_title")).\
limit(1).show(truncate=False)

+--------------------+--------------+
|Job Title           |Long Job Title|
+--------------------+--------------+
|Staff Data Scientist|20            |
+--------------------+--------------+



In [12]:
#100% number of remote workers in Germany and sort salary
df.where((col("company_location")=="DE")&(col("remote_ratio")==100)).\
orderBy("salary").select("experience_level","job_title","salary","remote_ratio","company_location").\
show()

+----------------+--------------------+------+------------+----------------+
|experience_level|           job_title|salary|remote_ratio|company_location|
+----------------+--------------------+------+------------+----------------+
|              EN|         ML Engineer| 14000|         100|              DE|
|              MI|      Data Scientist| 21600|         100|              DE|
|              EN|       Data Engineer| 48000|         100|              DE|
|              MI|Data Engineering ...| 51999|         100|              DE|
|              EN|       Data Engineer| 52800|         100|              DE|
|              EN|Data Science Cons...| 65000|         100|              DE|
|              SE|Data Analytics En...| 67000|         100|              DE|
|              MI|Machine Learning ...| 80000|         100|              DE|
|              EN|Machine Learning ...| 85000|         100|              DE|
|              SE|Principal Data Sc...|130000|         100|              DE|

In [13]:
#how many times which employee_residence is mentioned in the document and sort
df.groupBy("employee_residence").count().orderBy(F.desc("count")).select(col("employee_residence").alias("Employee Residence"),col("count").alias("Count")).show(100)

+------------------+-----+
|Employee Residence|Count|
+------------------+-----+
|                US|  332|
|                GB|   44|
|                IN|   30|
|                CA|   29|
|                DE|   25|
|                FR|   18|
|                ES|   15|
|                GR|   13|
|                JP|    7|
|                PT|    6|
|                PK|    6|
|                BR|    6|
|                NL|    5|
|                PL|    4|
|                IT|    4|
|                RU|    4|
|                TR|    3|
|                AT|    3|
|                VN|    3|
|                AU|    3|
|                AE|    3|
|                SG|    2|
|                HU|    2|
|                RO|    2|
|                NG|    2|
|                MX|    2|
|                SI|    2|
|                BE|    2|
|                DK|    2|
|                DZ|    1|
|                EE|    1|
|                UA|    1|
|                CN|    1|
|                HN|    1|
|

In [14]:
#where employee_residence="TR"
df.where(col("employee_residence")=="TR").select("work_year","experience_level","job_title","salary_in_usd","remote_ratio","employee_residence","company_location").show()

+---------+----------------+--------------+-------------+------------+------------------+----------------+
|work_year|experience_level|     job_title|salary_in_usd|remote_ratio|employee_residence|company_location|
+---------+----------------+--------------+-------------+------------+------------------+----------------+
|     2021|              MI| Data Engineer|        12103|           0|                TR|              TR|
|     2021|              MI| Data Engineer|        28016|         100|                TR|              TR|
|     2021|              SE|Data Scientist|        20171|          50|                TR|              TR|
+---------+----------------+--------------+-------------+------------+------------------+----------------+



In [15]:
#Find TRY/USD Ratio
df.withColumn("TRY/USD Ratio",col("salary")/col("salary_in_usd")).where(col("employee_residence")=="TR").select("work_year","TRY/USD Ratio").show()


+---------+-----------------+
|work_year|    TRY/USD Ratio|
+---------+-----------------+
|     2021| 8.92340741964802|
|     2021|8.923472301541976|
|     2021|8.923702344950671|
+---------+-----------------+



In [16]:
#how many times which currency is mentioned in the document and sort
df.groupBy("salary_currency").count().orderBy(F.desc("count")).select(col("salary_currency").alias("Salary Currency"),col("count").alias("Count")).show()

+---------------+-----+
|Salary Currency|Count|
+---------------+-----+
|            USD|  398|
|            EUR|   95|
|            GBP|   44|
|            INR|   27|
|            CAD|   18|
|            TRY|    3|
|            JPY|    3|
|            PLN|    3|
|            CNY|    2|
|            DKK|    2|
|            BRL|    2|
|            AUD|    2|
|            MXN|    2|
|            HUF|    2|
|            SGD|    2|
|            CHF|    1|
|            CLP|    1|
+---------------+-----+



In [17]:
#Find All Currency Ratio and sort according to salary_currency for 2022
df.withColumn("All Currency Ratio",col("salary")/col("salary_in_usd")).orderBy("salary_currency").where(col("work_year")==2022).select("work_year","salary_currency","All Currency Ratio").show()


+---------+---------------+------------------+
|work_year|salary_currency|All Currency Ratio|
+---------+---------------+------------------+
|     2022|            AUD|1.3840434658278524|
|     2022|            AUD|1.3840351544929241|
|     2022|            CAD|1.2691804901575052|
|     2022|            CAD|1.2691708253164042|
|     2022|            CAD| 1.269180853500267|
|     2022|            CAD|1.2691787003610109|
|     2022|            CAD|  1.26918195454021|
|     2022|            CAD|1.2691804901575052|
|     2022|            CAD|1.2691751207831656|
|     2022|            CHF| 0.939957170647181|
|     2022|            CNY| 6.445375443119562|
|     2022|            EUR|0.9097939316744758|
|     2022|            EUR|0.9097939316744758|
|     2022|            EUR|0.9098077273002972|
|     2022|            EUR|0.9098022089997635|
|     2022|            EUR|0.9097998440343125|
|     2022|            EUR|0.9098077273002972|
|     2022|            EUR|0.9098022089997635|
|     2022|  